In [66]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import keras
import keras_metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, SpatialDropout1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence,text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adamax
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [3]:
%cd MyDrive/Colab\ Notebooks

/gdrive/MyDrive/Colab Notebooks


In [4]:
!ls

'IMDB Dataset.csv'   NLP_Project_draft.ipynb   Untitled0.ipynb


In [46]:
data = pd.read_csv('IMDB Dataset.csv')

In [47]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [48]:
X = data['review']
y = data['sentiment']

Removing html and symbols from reviews

In [49]:
X = X.replace({'<.*?>': ''}, regex = True)
X = X.replace({'[^A-Za-z]': ' '}, regex = True)

In [ ]:
X[0]

Removing stop words and tokenizing

In [51]:
stop_words = set(stopwords.words('english'))
for i in range(len(X)):
  X[i]=word_tokenize(X[i])
  X[i] = [w for w in X[i] if not w.lower() in stop_words]

Encoding sentiment

In [53]:
y = y.replace('positive', 1)
y = y.replace('negative', 0)

Split into 80:20 ratio 

In [55]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [56]:
unique_word = len(np.unique(x_train))
print(unique_word)

Calculating maximum length of the given reviews

In [58]:
length = []
for i in x_train:
  length.append(len(i))
max_length = int(np.ceil(np.mean(length)))
print(max_length)

119


Tokenizing and padding for model input

In [59]:
token = Tokenizer(lower=False) 
token.fit_on_texts(x_train)

x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

In [85]:
total_words = len(token.word_index) + 1

In [60]:
x_train

array([[3136,  983,  357, ...,    0,    0,    0],
       [ 318, 4109,  459, ...,    0,    0,    0],
       [  13,  105,    2, ...,    0,    0,    0],
       ...,
       [ 158,  327, 3293, ...,   36,  438, 3169],
       [1661,  279,  473, ...,    0,    0,    0],
       [  92,   45,  210, ..., 1406,   24,   92]], dtype=int32)

CNN model

In [68]:
model = Sequential()
model.add(Embedding(unique_word, 150, input_length = max_length))
model.add(Conv1D(32,kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.summary()
model.add(Dense(2, activation='sigmoid'))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 119, 150)          5961300   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 119, 32)           14432     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 59, 32)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 59, 64)            6208      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 29, 64)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 29, 64)            12352     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 14, 64)           

In [75]:
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])

In [84]:
print(x_test.shape, y_test.shape)

(10000, 119) (10000,)


In [ ]:
#model.fit(x_train, y_train, batch_size = 128, epochs = 5)

LSTM model

In [86]:
model1 = Sequential()
model1.add(Embedding(total_words, 32, input_length = max_length))
model1.add(LSTM(64))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer = 'adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])
print(model1.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 119, 32)           3748192   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 3,773,089
Trainable params: 3,773,089
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#model.fit(x_train, y_train, batch_size = 128, epochs = 10)